In [5]:
import nltk
import pickle
import argparse
from collections import Counter
from glob import glob
from tqdm import tqdm
nltk.download('punkt')
import json
import time

class Vocabulary:
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0

    def add_word(self, word):
        if word not in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1

    def __call__(self, word):
        if word not in self.word2idx:
            return self.word2idx['<unk>']
        return self.word2idx[word]

    def __len__(self):
        return len(self.word2idx)

def build_vocab(json_list, threshold):
    """Build a simple vocabulary wrapper."""
    
    
    counter = Counter()  # 중복 제거

    for i in tqdm(range(len(json_list))):
        with open(json_list[i], 'r', encoding='utf-8-sig') as file:
            data = json.load(file)
        caption = str(data['content']['file']['patch_discription'])
        tokens = nltk.tokenize.word_tokenize(caption.lower())

        counter.update(tokens)

        if (i + 1) % 10000 == 0:
            print("[{}/{}] Tokenized the captions.".format(i + 1, len(json_list)))

    # If the word frequency is less than 'threshold', discard the word
    words = [word for word, cnt in counter.items() if cnt >= threshold]

    # Create a vocab wrapper and add some special tokens
    vocab = Vocabulary()
    vocab.add_word('<pad>')
    vocab.add_word('<start>')
    vocab.add_word('<end>')
    vocab.add_word('<unk>')

    # Add the words to the vocabulary
    for word in words:
        vocab.add_word(word)

    return vocab


[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
start_time = time.time()
print("Start Time:", time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(start_time)))
json_list=glob('../../data/synth/010.위암 병리 이미지 및 판독문 합성 데이터/1.데이터/**/2.라벨링데이터/**/*.json')
vocab=build_vocab(json_list, 4)
vocab_path = '../../data/synth/010.위암 병리 이미지 및 판독문 합성 데이터/1.데이터/vocab.pkl'
with open(vocab_path, 'wb') as f:
    pickle.dump(vocab, f)
print("Total vocabulary size: {}".format(len(vocab)))
print("Saved the vocabulary wrapper to '{}'".format(vocab_path))
end_time = time.time()
print("End Time:", time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(end_time)))
elapsed_time = end_time - start_time
print(f"Elapsed Time: {elapsed_time:.2f} seconds")

Start Time: 2024-11-10 13:54:11


100%|██████████| 10000/10000 [20:09<00:00,  8.27it/s]

[10000/10000] Tokenized the captions.
Total vocabulary size: 83
Saved the vocabulary wrapper to '../../data/synth/010.위암 병리 이미지 및 판독문 합성 데이터/1.데이터/vocab.pkl'
End Time: 2024-11-10 14:14:21
Elapsed Time: 1209.95 seconds


In [7]:

json_list[0]

'../../data/synth/010.위암 병리 이미지 및 판독문 합성 데이터/1.데이터/1.Training/2.라벨링데이터/미만형선암/NIA6_S_STDI_01870.json'